Scrape the necessary information from the websites provided

In [1]:
# Dependencies
from splinter import Browser
from bs4 import BeautifulSoup as bs
import requests
import time
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
# Configure settings for splinter
executable_path = {"executable_path": ChromeDriverManager().install()}
browser = Browser("chrome", **executable_path, headless=False)

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Trying to download new driver from http://chromedriver.storage.googleapis.com/87.0.4280.20/chromedriver_win32.zip


[WDM] - Driver has been saved in cache [C:\Users\ramya\.wdm\drivers\chromedriver\win32\87.0.4280.20]


In [5]:
# Part 1: NASA Mars News
# Visit mars.nasa.gov/news site
# Get html and parse it from mars events website using requests
url = "https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"
browser.visit(url)
# Scrape page into Soup
html = browser.html
soup = bs(html, "html.parser")

In [6]:
# Part 1: NASA Mars News- Find most recent news article title and description
a = soup.find("body", id="news")
b = a.find("ul",class_="item_list")
c = b.find("li",class_="slide")
d = c.find("div",class_="image_and_description_container")
e = d.find("div",class_="list_text")
f = e.find("div",class_="content_title")

news_title = f.find("a").text
news_description = e.find("div",class_="article_teaser_body").text

print(news_title)
print(news_description)

MOXIE Could Help Future Rockets Launch Off Mars
NASA's Perseverance rover carries a device to convert Martian air into oxygen that, if produced on a larger scale, could be used not just for breathing, but also for fuel.


In [7]:
# Part 2: JPL Mars Space Images Featured Image - Get html and parse it
url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(url)
# Scrape page into Soup
html = browser.html
soup = bs(html, "html.parser")

In [8]:
# Part 2: JPL Mars Space Images Featured Image - Find the featured image url

# Click on FULL IMAGE button
link1 = browser.find_by_tag('a').links.find_by_partial_text("FULL IMAGE")
link2 = link1.click()
time.sleep(10)

# Click on More Info button
link3 = browser.links.find_by_partial_text("more info")
link3.click()
time.sleep(10)

#Save full size of featured image 
link4 = browser.find_by_tag('figure[class="lede"]')
link5 = link4.find_by_tag('a')
for image in link5:
    featured_image_url = image["href"]

featured_image_url

'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA20057_hires.jpg'

In [9]:
# Part 3: Mars Facts
#Scrape page into soup
url = "https://space-facts.com/mars/"
browser.visit(url)
html = browser.html
soup = bs(html, "html.parser")  

In [10]:
# Part 3: Mars Facts (Contd.)
a = soup.find("table",id="tablepress-p-mars-no-2")
b = a.find_all("tr")
metric = list()
value = list()
for row in b:
    metric.append(row.find("td",class_="column-1").text)
    value.append(row.find("td",class_="column-2").text)

mars_facts = pd.DataFrame({'Metric': metric,
                           'Value' : value}, 
                          columns = ['Metric','Value'])
mars_facts.set_index('Metric')
# convert dataframe to html
mars_facts_html = mars_facts.to_html()
mars_facts_html

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Metric</th>\n      <th>Value</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>5</th>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>6</th>\n      <td>Surface Temperature:</td>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>7</th>\n      <td>First Record:</td>\n      <td>2nd millennium BC</t

In [89]:
# Part 4: Mars Hemispheres

#Scrape page into soup
url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url)
html = browser.html
soup = bs(html, "html.parser")  

# Find the body of the page that includes the necessary links for the hemispheres
link1 = browser.find_by_tag('div[id="product-section"]')
h_count = 0
title = list()
img_url = list()


# Within the div: product-section, there are 4 class items. Each one corresponds to a different hemisphere 
while h_count < 4:
    link2 = link1.find_by_tag('div[class="item"]')[h_count]
    link3 = link2.find_by_tag("a")
    link4 = link3.last
    link5 = link4.click()
    time.sleep(1)
    # Get the hemisphere name, store in a list
    hlink1 = browser.find_by_tag('section[class="block metadata"]')
    #title.append(hlink1.find_by_tag('h2[class="title"]').text)
    
    # Get the full size image url for each hemisphere
    hlink2 = browser.find_by_tag('div[id="wide-image"]')
    hlink3 = hlink2.find_by_tag('img[class="wide-image"]')
    #img_url.append(hlink3['src'])
    
    # Go back to the main page and start again for the next hemisphere
    link6 = browser.back()
    link1 = browser.find_by_tag('div[id="product-section"]')
    h_count +=1
title, img_url
# Save as a dictionary
mars_facts = pd.DataFrame({'Metric': metric,
                           'Value' : value}, 
                          columns = ['Metric','Value'])
mars_facts.set_index('Metric')

(['Cerberus Hemisphere Enhanced',
  'Schiaparelli Hemisphere Enhanced',
  'Syrtis Major Hemisphere Enhanced',
  'Valles Marineris Hemisphere Enhanced'],
 ['https://astrogeology.usgs.gov/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg',
  'https://astrogeology.usgs.gov/cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg',
  'https://astrogeology.usgs.gov/cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg',
  'https://astrogeology.usgs.gov/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'])